In [2]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 4.3 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630156 sha256=1f81a77e0727cf6971c8c1ecb5c02b8c5a2c5ea24681606b9e5983dcf5736fcf
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [8]:
!pip install pickle5

     |████████████████████████████████| 256 kB 4.3 MB/s 


In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
from tqdm import trange, notebook
import pickle5 as pickle
from surprise import SVD
from surprise.model_selection import cross_validate

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/Othercomputers/내 노트북/07_PJT_final_01/02_style_recommender_system

/content/drive/Othercomputers/내 노트북/07_PJT_final_01/02_style_recommender_system


In [4]:
with open('./df3.pkl', "rb") as fh:
  data = pickle.load(fh)

In [5]:
data.columns

Index(['name', 'main_style', 'sub_styles', 'cloth_cat', 'color', 'pattern',
       'length', 'arm_length', 'fit_cloth', 'fabric'],
      dtype='object')

In [ ]:
data.main_style.unique()

array(['공', '기타', '사', '운동'], dtype=object)

항목기반 :  스타일 / 유저기반 : 파일명 → 두 값은 중복이 없어야 함

- 사용자가 선택한 상황별, 날씨 별 선택지에 따라(데이터셋의 항목들 활용) 가중치를 주어 이를 기반으로 데이터베이스 구축
    - 컨텐츠 기반 필터링 사용 ( 가중치 : rating, 항목 : 스타일, 유저: 파일명)
    - 파일명 기준으로 선발
    - 파일명 내부 의류 카테고리 확인
    - 사용자 옷장 속 db와 비교
        - 옷장에 있을 경우 선호도 반영하여 우선 출력
        - 옷장에 없을 경우 부족한 항목 별도 출력
- 사용자가 옷 이미지 입력 시 입력한 선호도를 반영하여 1차 확인한 순서 중 우선 순위 추출

코디 → 사용자 설문 → 선택한 항목에 따라 가중치 반영 → [코드 활용 가능](https://github.com/gjtjdtn201/ShawcheckRedemption/blob/ft_front/backend/hss/wear/coordiset.py)

- 상황별 선택지
    - 운동(스포티)
    - 공(매니시, 모던, 소피스트케이티드, 아방가르드, 젠더리스, 클래식, 톰보이)
    - 사(로맨틱, 스트리트, 웨스턴, 컨트리, 펑크, 페미닌, 프레피, 기타)
    - 기타(레트로, 오리엔탈, 키치, 히피)
- 날씨(자동반영, 추가선택가능)
    - 날씨에 따른 기장 필터링
        - length : ['발목', None, '하프', '미니', '노멀', '롱', '맥시', '노말', '크롭', '미디', '니렝스’]
        - arm_length : [None, '긴팔', '7부소매', '없음', '반팔', '민소매', '캡’]
    - 날씨에 따른 패턴 필터링
        - 날이 화창한 날 사적인 상황 : 플로랄 같은 화사한 유형에 가중치 부여

In [ ]:
# 상황별 가중치 부여 
# sport = ['스포티']
# public_cloth = ['매니시', '모던', '소피스트케이티드', '아방가르드', '젠더리스', '클래식', '톰보이']
# privacy_cloth = ['로맨틱', '스트리트', '웨스턴', '컨트리', '펑크', '페미닌', '프레피', '기타']
# other_cloth = ['레트로', '오리엔탈', '키치', '히피']

In [7]:
# 컨텐츠기반필터링에 내보낼 데이터베이스 형태 
# coulmns = [filename, style_id, rating]

# 상황 
# sport : 스포츠
# public : 공적인 일
# privacy : 사적인 일
# other : 기타(여행과 같은 이벤트성 있는 상황)
select = int(input('1. 공적인 일\n2. 사적인 일\n3. 운동\n4. 기타(여행과 같은 이벤트성 상황)\n\n위 항목 중 필요한 상황을 번호로 선택해주세요. :'))

# 선택한 메인 스타일만 가져옴 '공', '기타', '사', '운동'
if select == 1:
    condition1 = (data['main_style'] == '공')
elif select == 2:
    condition1 = (data['main_style'] == '사')
elif select == 3:
    condition1 = (data['main_style'] == '운동')
elif select == 4:
    condition1 = (data['main_style'] == '기타')
else:
    print('1 - 4 번 상황 중 선택해주세요')

data_public = data[condition1]

1. 운동
2. 공적인 일
3. 사적인 일
4. 기타(여행과 같은 이벤트성 상황)

위 항목 중 필요한 상황을 번호로 선택해주세요. :2


In [8]:
# 온도별 소재 선택
temp = int(input('오늘의 온도를 입력하세요.'))

# 선택한 메인 스타일만 가져옴 '공', '기타', '사', '운동'
if temp < 5:
    fabric_list = ['가죽', '울캐시미어', '니트', '퍼', '패딩', '벨벳', '코듀로이','플리스', '무스탕', '자카드', '데님','스웨이드','헤어']
    cloth_cat_list = ['상의', '하의','아우터', '원피스']
    arm_length_list = ['긴팔']    
elif temp in range(5,9):
    fabric_list = ['데님', '니트', '스웨이드', '플리스', '트위드','울캐시미어','퍼', '벨벳', '스웨이드','코듀로이','헤어']
    cloth_cat_list = ['상의', '하의','아우터', '원피스']
    arm_length_list = ['긴팔']
elif temp in range(9,12):
    fabric_list = ['우븐', '가죽', '데님', '니트', '벨벳', '스웨이드','코듀로이','트위드','헤어']
    cloth_cat_list = ['상의', '하의','아우터', '원피스']
    arm_length_list = ['긴팔']
elif temp in range(12,17):
    fabric_list = ['우븐', '데님', '시폰', '레이스', '실크', '니트','시폰']
    cloth_cat_list = ['상의', '하의','아우터', '원피스']
    arm_length_list = ['긴팔']
elif temp in range(17,20):
    fabric_list = ['우븐', '가죽', '데님', '네오프렌','실크','시폰']
    cloth_cat_list = ['상의', '하의','아우터', '원피스']
    arm_length_list = ['7부소매','긴팔','반팔']
elif temp in range(20,23):
    fabric_list = ['린넨', '데님','시폰']
    cloth_cat_list = ['상의', '하의','아우터', '원피스']
    arm_length_list = ['7부소매','긴팔','반팔']
elif temp in range(23,28):
    fabric_list = ['린넨', '데님','메시','스판덱스']
    cloth_cat_list = ['상의', '하의', '원피스']
    arm_length_list = ['민소매','반팔','캡']
elif temp >= 28:
    fabric_list = ['린넨', '데님','메시','스판덱스']
    cloth_cat_list = ['상의', '하의', '원피스']
    arm_length_list = ['민소매','반팔','캡']
else:
    print('잘못된 온도를 입력하셨습니다. 다시 입력해주세요.')

data2 = data[data['fabric'].map(lambda x: any(fabric in x for fabric in fabric_list))]
data2[condition1 & data2['cloth_cat'].isin(cloth_cat_list) & data2['arm_length'].isin(arm_length_list)]

오늘의 온도를 입력하세요.2


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,name,main_style,sub_styles,cloth_cat,color,pattern,length,arm_length,fit_cloth,fabric
255,101297,공,매니시,아우터,브라운,무지,하프,긴팔,루즈,가죽
257,101298,공,매니시,아우터,브라운,무지,하프,긴팔,루즈,가죽
259,101299,공,매니시,아우터,브라운,무지,하프,긴팔,루즈,가죽
261,101300,공,매니시,아우터,브라운,무지,하프,긴팔,루즈,가죽
263,101301,공,매니시,아우터,브라운,무지,하프,긴팔,루즈,가죽
...,...,...,...,...,...,...,...,...,...,...
298688,966846,공,톰보이,아우터,베이지,,하프,긴팔,루즈,플리스 우븐
298691,966847,공,톰보이,아우터,베이지,,하프,긴팔,루즈,플리스 우븐
298694,966848,공,톰보이,아우터,베이지,,하프,긴팔,루즈,플리스 우븐
298697,966849,공,톰보이,아우터,베이지,,하프,긴팔,루즈,플리스 우븐


In [9]:
public_list=['남성적', '중성적', '우아함', '산뜻함','도시적', '미래지향적', '모던함','세련된', '도시적', '지적', '여성스러운','독특한', '혁신적', '중성적인', '개성있는', '편견없는', '고전적', '우아한','남성적', '편안한', '단순한', '현대적인']
other_list=['복고풍', '레트로','화려한', '고급스러운', '다채로운', '고전적인','장식이과한',  '자유분방한','자유로운', '세련된', '화려한', '슬림한']
privacy_list=['여성스러운', '귀여운', '사랑스러운', '소녀적인','현대적인', '자유로운', '힙한','도시적', '세련된', '야성적','내추럴한', '편한', '소박한', '소녀적인', '반항적인', '자유분방한', '독특한', '여성스러운', '부드러운', '러블리한', '차분한','캐주얼한', '단정한', '세련된']
# sport_list=['활동적인', '편안한', '자연스러운'] # 스타일이 스포티 하나만 있으므로 스포츠 선택 시 추가 선택 제외

In [ ]:
a = list(input(f'{public_list}'))

In [ ]:

if select == 1: # public
    list(input(f''))
elif select == 2:   # privacy
elif select == 4:   # sport


In [ ]:
매니시=['남성적', '중성적', '우아함', '산뜻함']
모던=['도시적', '미래지향적', '모던함']
소피스트케이티드=['세련된', '도시적', '지적', '여성스러운']
아방가르드=['독특한', '혁신적']
젠더리스=['중성적인', '개성있는', '편견없는']
클래식=['고전적', '세련된', '우아한']
톰보이=['남성적', '편안한', '단순한', '현대적인']

레트로=['복고풍', '레트로']
오리엔탈=['화려한', '고급스러운', '다채로운', '고전적인'] 
키치=['장식이과한',  '자유분방한']
히피=['자유로운', '세련된', '화려한', '슬림한']

로맨틱=['여성스러운', '귀여운', '사랑스러운', '소녀적인']
스트리트=['현대적인', '자유로운', '힙한']
웨스턴=['도시적', '세련된', '야성적']
컨트리=['내추럴한', '편한', '소박한', '소녀적인']
펑크=['반항적인', '자유분방한', '독특한']
페미닌=['여성스러운', '부드러운', '러블리한', '차분한']
프레피=['캐주얼한', '단정한', '세련된']

스포티=['활동적인', '편안한', '자연스러운']

In [ ]:
기상API : 기온, 강수, 바람

봄 3-5월
- 겨울같은 봄 : 5 - 15 도 사이
- 꽃놀이/피크닉/소풍 가고 싶은 봄 : 10 - 20도
- 아직봄인데여름같은 봄 : 15 - 25도

여름 6-9월 초 -> 추가로 습도
- 끈적끈적한 여름 : 
- 여행가고싶은여름 : 
- 장마철 여름 : 기간 중 비가 올때

가을 9월 중순-11월
- 겨울이 온 것 같은 가을
- 트렌치코트가 생각나는 가을
- 단풍구경이 생각나는 가을 

겨울 12월 - 2월
- 포근한 겨울
- 집밖에나가면안되는겨울 : 한파예보
- 함박눈내리는 겨울 : 기간 중 눈이 예보되어있을때



In [ ]:
# 소재, 소매기장, 기장으로 판별
# - length
#     - 상의기장 : 크롭, 노멀, 롱
#     - 하의기장 : 미니, 니렝스, 미디, 발목, 맥시
#     - 아우터기장 : 크롭, 노멀, 하프, 롱, 맥시
#     - 원피스 기장 : 미니, 니렝스, 미디, 발목, 맥시
# - arm_length
#     - 소매기장 : 민소매, 반팔, 캡, 7부소매, 긴팔
# - fit
#     - 상의, 아우터, 원피스 : 타이트, 노멀, 루즈, 오버사이즈
#     - 하의 : 스키니, 노멀, 와이드, 루즈, 벫보텀
# - (new) fabric : unique로 확인하여 필요한 값(특정 계절)만 추출
# '우븐', '린넨', '가죽', '저지', '울캐시미어', '데님', '니트', '실크', '퍼', '패딩',
# '시폰', '스판덱스', '벨벳', '스웨이드', '코듀로이', '플리스', '레이스', '트위드', '헤어',
# '메시', '자카드', '무스탕', '네오프렌', '비닐', '시퀸글리터'


    